In [ ]:
import os, sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *
from nba_api.stats.endpoints import leaguedashteamstats


fig_DIR = "../figs/team_ratings/"
box_DIR = "../data/box/"

In [ ]:
cols1 = ['TEAM_NAME', 'GP', 'W', 'L', 'W_PCT',
       'OFF_RATING', 'DEF_RATING', 'NET_RATING', 'EFG_PCT', 'TS_PCT', 'E_PACE', 'PACE', 'POSS',
       "W_RANK", "L_RANK",
       'W_PCT_RANK', 'OFF_RATING_RANK', 'DEF_RATING_RANK',
       'NET_RATING_RANK',]

In [ ]:
dfa = []
for year in range(2001,2023):
    stats = leaguedashteamstats.LeagueDashTeamStats(
        measure_type_detailed_defense="Advanced",
        season = get_ss(year)
    )
    df1 = stats.get_data_frames()[0]
    df1 = df1.sort_values("W_PCT",ascending=False).reset_index(drop=True)
    time.sleep(0.6)
    stats = leaguedashteamstats.LeagueDashTeamStats(
        measure_type_detailed_defense="Advanced",
        season_type_all_star="Playoffs",
        season=get_ss(year)
    )
    df2 = stats.get_data_frames()[0]
    df2 = df2.sort_values("W",ascending=False).reset_index(drop=True)
    df1 = df1[cols1]
    df2 = df2[cols1]
    df3 = pd.merge(df2,df1,on=["TEAM_NAME"])
    df3["season"] = year + 1
    
    time.sleep(0.6)
    stats = leaguegamelog.LeagueGameLog(
        player_or_team_abbreviation="T",
        season=get_ss(year),
        season_type_all_star="Playoffs",
    )
    df4 = stats.get_data_frames()[0]
    df4["Round"] = df4["GAME_ID"].str[-3]
    df4["Win"] = df4["WL"] == "W"
    df4["Loss"] = df4["WL"] == "L"
    df5 = df4.groupby(["TEAM_NAME","Round"])[["Win","Loss"]].sum().reset_index()
    df6 = df5.groupby(["TEAM_NAME"])[["Round"]].count().reset_index()
    df7 = pd.merge(df3,df6,on="TEAM_NAME")
    dfa.append(df7)
    time.sleep(0.6)
df8 = pd.concat(dfa)
df8 = df8.rename(columns={"Round":"Round_x"})

In [ ]:
df8.columns

In [ ]:
cols = ['GP_y', 'W_y', 'L_y', 'W_PCT_y', 'OFF_RATING_y',
       'DEF_RATING_y', 'NET_RATING_y',]
for col in cols:
    r2 = (df8["PACE_y"].corr(df8[col]))**2
    print(f"{col[:-2]} : {r2:0.3f}")

In [ ]:
x_p = df8["W_y"]
y_p = df8["W_x"]
slope, intercept, r, p, sterr = scipy.stats.linregress(x=x_p, y=y_p)
r2 = r**2
r2 = round(r2,3)
r2

In [ ]:
p = (
    ggplot(df8,aes(x="W_y",y="W_x"))
    + geom_point()
    + geom_smooth(method="lm",color="red",size=3)
    + labs(
        title="Regular Season Net Rating vs # of Playoff Wins",
        subtitle="From 1996-2023",
        caption="@SravanNBA | source:nba.com/stats",
        x="Regular Season Net Rating",
        y="# of Playoff Wins"
    )
    + annotate('label', x=30, y=15, label=f'r^2={r2}', size=14)
    + theme_idv
    + theme(
        figure_size=(5*1.1,4*1.1)
    )
)
p

In [ ]:
x_var = "NET_RATING_y"
y_var = "W_x"
fig = px.scatter(df8, x=x_var, y=y_var,width=800, height=600, hover_name="TEAM_NAME",hover_data=["season","NET_RATING_y","W_x","NET_RATING_x"],color_discrete_sequence =["red"])
# fig_line = px.line(df, x = "RAPM", y = "RAPM",color_discrete_sequence=["black"])
# fig = go.Figure(data = fig_10.data + fig_line.data)
fig.update_layout( width=800, height=600, showlegend=True)
fig.update_layout( title=dict(
                        text="Regular Season Net Rating vs # of Playoff Wins",
                        xanchor="auto",
                        yanchor="middle",
                        font=dict(family="Arial, Tahoma, Helvetica", size=25, color="Black")
                    ),
            )
fig.update_xaxes(title_text=x_var)
fig.update_yaxes(title_text=y_var)
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))
fig.show()

## Playoff Round Wins

In [ ]:
qus = ["Round_x > 0","Round_x > 1","Round_x > 2","Round_x > 3","Round_x > 1 & W_RANK_x == 1"]
scenario = ["Playoffs","2nd Round","Conf Finals","Finals","Championship"]

In [ ]:
df88 = df8.query("season > 2017")

In [ ]:
dfb = []
for q,s in zip(qus,scenario):
    df9 = df88.query(q)
    df10 = df9[["OFF_RATING_RANK_y","DEF_RATING_RANK_y","NET_RATING_RANK_y"]].mean().round(2).reset_index().T
    df10.columns = ["Off Rtf Rank","Def Rtg Rank","Net Rtg Rank"]
    df10 = df10.iloc[1:]
    df10["Scenario"] = s
    dfb.append(df10)
df11 = pd.concat(dfb)

In [ ]:
df11.insert(0,"Scenario",df11.pop("Scenario"))

In [ ]:
df11 = df11.reset_index(drop=True)

In [ ]:
%reload_ext rpy2.ipython

In [ ]:
%%R -i df11
df <- df11
library(tidyverse)
library(gt)
# library(gtExtras)
df %>% 
  gt()%>%
  tab_header(
    title = md("**Average Regular Season Ratings Rankings vs Playoff Progress**"),
    subtitle = md("Last 5 Seasons"),
    ) %>%
    cols_align("center")%>%
    cols_align(align = "left",columns="Scenario")%>%
    tab_options(
        table.background.color = "floralwhite",
        column_labels.font.size = 12,
        column_labels.font.weight = 'bold',
        row_group.font.weight = 'bold',
        row_group.background.color = "#E5E1D8",
        table.font.size = 10,
        heading.title.font.size = 14,
        heading.subtitle.font.size = 12.5,
        table.font.names = "Consolas", 
        # data_row.padding = px(2),
        table.margin.left = 1,
        table.margin.right = 1,
    ) %>% 
    tab_source_note(
    source_note = "@SravanNBA | Source: nba.com/stats" )%>% gtsave("../figs/team_ratings/reg_ranking_vs_playoffs_prog.png",size=5) 